In [1]:
import popexposure as pe
import pandas as pd
import geopandas as gpd
import numpy as np
import math
from IPython.display import Markdown as md

In [2]:
wfbz_all = gpd.read_file('../data/raw/wfbz_disasters_2000-2025.geojson')
wfbz = wfbz_all[~wfbz_all.geometry.is_empty].copy()
wfbz['ID_hazard'] = wfbz['wildfire_id']
wfbz['buffer_dist_10km'] = 10_000
wfbz = {group_name: group_data for group_name, group_data in wfbz.groupby('wildfire_year')}
pop = {
  2000 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2000/GHS_POP_E2000_GLOBE_R2023A_54009_100_V1_0.tif'),
  2005 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2005/GHS_POP_E2005_GLOBE_R2023A_54009_100_V1_0.tif'),
  2010 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2010/GHS_POP_E2010_GLOBE_R2023A_54009_100_V1_0.tif'),
  2015 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2015/GHS_POP_E2015_GLOBE_R2023A_54009_100_V1_0.tif'),
  2020 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2020/GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0.tif'),
}


In [3]:
wfbz[2025]

,wildfire_id,wildfire_year,wildfire_states,wildfire_counties,wildfire_counties_fips,wildfire_area,wildfire_complex,wildfire_complex_names,wildfire_total_fatalities,wildfire_max_civil_fatalities,...,geometry_src,geometry_method,redbook_id,ics_id,fired_id,mtbs_id,fema_id,geometry,ID_hazard,buffer_dist_10km
2237,2238.0,2025.0,CA,LOS ANGELES,06037,57.251273,False,EATON,NaN,NaN,...,NIFC,NIFC by Name/Place/Time,None,2025_009087_EATON,None,None,FM-5550-CA,"MULTIPOLYGON (((-118.09858 34.22913, -118.0986...",2238.0,10000
2238,2239.0,2025.0,CA,LOS ANGELES,06037,42.197125,False,HUGHES|HURST,NaN,NaN,...,NIFC,NIFC by Name/Place/Time,None,2025_0003294_HURST|2025_0003294_HURST|2025_250...,None,None,FM-5551-CA,"POLYGON ((-118.46974 34.33082, -118.46984 34.3...",2239.0,10000
2239,2240.0,2025.0,CA,LOS ANGELES,06037,94.920288,False,PALISADES,NaN,NaN,...,NIFC,NIFC by Name/Place/Time,None,2025_000738_PALISADES,None,None,FM-5549-CA,"MULTIPOLYGON (((-118.56087 34.0441, -118.56084...",2240.0,10000
5944,5945.0,2025.0,ND,GRANT,38037,16.025566,False,HWY 49,NaN,NaN,...,ICS209,"ICS by Point of Origin, Size",None,2025_000035_HWY 49,None,None,None,"POLYGON ((-101.88087 46.69586, -101.88125 46.6...",5945.0,10000
5945,5946.0,2025.0,PA,BERKS,42011,0.004047,False,LAMMS HL,NaN,NaN,...,ICS209,"ICS by Point of Origin, Size",None,2025_000142_LAMMS HILL,None,None,None,"POLYGON ((-76.09002 40.38329, -76.09001 40.383...",5946.0,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6111,6112.0,2025.0,VA,FRANKLIN,51067,2.246007,False,CAHAS MOUNTTAIN,NaN,NaN,...,ICS209,"ICS by Point of Origin, Size",None,2025_25WR00032_CAHAS MOUNTTAIN FIRE,None,None,None,"POLYGON ((-80.03079 37.11383, -80.03061 37.113...",6112.0,10000
6112,6113.0,2025.0,VA,WASHINGTON,51191,0.728435,False,LIVINGSTON CRK,NaN,NaN,...,ICS209,"ICS by Point of Origin, Size",None,2025_25WR00160_LIVINGSTON CREEK FIRE,None,None,None,"POLYGON ((-82.29639 36.67643, -82.29643 36.676...",6113.0,10000
6113,6114.0,2025.0,VA,CARROLL,51035,0.004047,False,POT ROCK RD,NaN,NaN,...,ICS209,"ICS by Point of Origin, Size",None,2025_25WR00358_POT ROCK ROAD FIRE,None,None,None,"POLYGON ((-80.9153 36.76689, -80.9153 36.76689...",6114.0,10000
6114,6115.0,2025.0,VA,GRAYSON,51077,0.003035,False,VIS RD,NaN,NaN,...,ICS209,"ICS by Point of Origin, Size",None,2025_25WR00464_VISTA ROAD FIRE,None,None,None,"POLYGON ((-81.07202 36.678, -81.07202 36.67801...",6115.0,10000


In [4]:
pop_impacted = dict()
for yr in range(2000,2026):
  print(yr)
  pop_impacted[yr] = pop[min(math.floor(yr/5)*5, 2020)].est_exposed_pop(
    hazard_data=wfbz[yr],
    hazard_specific=False
  )

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025


In [5]:
pop_impacted_all = pd.concat(pop_impacted.values(), ignore_index=True)
pop_impacted_all['year'] = range(2000,2026)

In [10]:
md(f"""Studies document increasing wildfire smoke exposure in the United States, but no
research has characterized the locations of wildfires that impact human populations (i.e.,
wildfire burn zone disasters [WFBZ disasters]). Here, we leverage a novel dataset of WFBZ
disasters by harmonizing six existing national or California-specific datasets to
characterize spatiotemporal trends in WFBZ disaster occurrence, size, and severity,
populations impacted, intersection with the wildland-urban interface, and FEMA disaster
declarations. Between 2000-2025, we identified {len(wfbz_all):,.0f} WFBZ disasters in the United States.
These WFBZ disasters burned a cumulative {(wfbz_all.to_crs(3857).dissolve().area/1000/1000).iloc[0]:,.0f} km<sup>2</sup>, with the burned area exceeding
20,000km<sup>2</sup> in Texas, Idaho, Oregon, and California. On average, {pop_impacted_all.exposed_10km.mean():,.0f} people lived within 10km
km of a WFBZ annually, with a {(100*((pop_impacted_all.exposed_10km / pop_impacted_all.exposed_10km.shift(1)).mean() - 1)):,.0f}% average increase in count annually during the study
period. California and Hawaii WFBZ disasters accounted for 67% (n=306) of civilian
fatalities and California alone accounted for the majority of structures destroyed ({(100*(wfbz_all[wfbz_all['wildfire_states']=='CA'].wildfire_struct_destroyed.sum()/wfbz_all.wildfire_struct_destroyed.sum())):,.0f}%,
n={(wfbz_all[wfbz_all['wildfire_states']=='CA'].wildfire_struct_destroyed.sum()):,.0f}). We find that certain states were more likely to receive a FEMA disaster declaration
for WFBZ disasters. For example, Hawaii and New Hampshire received FEMA disaster
declarations for >85% of identified WFBZ disasters, while California, Montana, and Texas
received one for 20-30% of WFBZ disasters. Understanding trends in WFBZ locations,
severity, and populations exposed can aid in disaster preparedness and recovery efforts.""")

Studies document increasing wildfire smoke exposure in the United States, but no
research has characterized the locations of wildfires that impact human populations (i.e.,
wildfire burn zone disasters [WFBZ disasters]). Here, we leverage a novel dataset of WFBZ
disasters by harmonizing six existing national or California-specific datasets to
characterize spatiotemporal trends in WFBZ disaster occurrence, size, and severity,
populations impacted, intersection with the wildland-urban interface, and FEMA disaster
declarations. Between 2000-2025, we identified 6,770 WFBZ disasters in the United States.
These WFBZ disasters burned a cumulative 409,478 km<sup>2</sup>, with the burned area exceeding
20,000km<sup>2</sup> in Texas, Idaho, Oregon, and California. On average, 4,907,766 people lived within 10km
km of a WFBZ annually, with a 18% average increase in count annually during the study
period. California and Hawaii WFBZ disasters accounted for 67% (n=306) of civilian
fatalities and California alone accounted for the majority of structures destroyed (68%,
n=87,183). We find that certain states were more likely to receive a FEMA disaster declaration
for WFBZ disasters. For example, Hawaii and New Hampshire received FEMA disaster
declarations for >85% of identified WFBZ disasters, while California, Montana, and Texas
received one for 20-30% of WFBZ disasters. Understanding trends in WFBZ locations,
severity, and populations exposed can aid in disaster preparedness and recovery efforts.